В Scikit-learn к сожалению нет готовых методов для генерации рекомендаций (считается, что их можно получить базовым функционалом библиотеки). Однако существует несколько популярных библиотек, упрощающих разработку рекомендательных систем. 

В этом упражнении рассмотрим три такие билиотеки: **lightFM**, **Surprise** и **Implicit**

# <span style="color:blue">LightFM<span>

Установка:
- pip install lightfm
- conda install -c conda-forge lightfm
- через GUI анаконды (проверить, что добавлен channel "conda-forge")

В библиотеке есть 2 встроенных датасета: 
- Movielens (оценки фильмов по шкале 1-5)
- Stackexchange

Будем работать с MovieLens:

In [129]:
from lightfm.datasets import fetch_movielens
data = fetch_movielens()

In [ ]:
# Названия фильмов
titles = data['item_labels']

# Оценки фильмов (обучающая выборка)
ratings = data['train'].tocsr()

# Оценки фильмов (тестовая выборка)
ratings_test = data['test'].tocsr()

Данные выгружаются в разреженном формате COO, поэтому для удобства можем перевести в формат CSR (для этого есть метод tocsr()).


Проставленные оценки пользователя

Есть два распространенных способа представления разреженных матриц COO и CSR:

- COO - три списка:
    - ptr (номер эемента col, с которого отсчитывается новая строка)
    - col (номера столбцов)
    - data (данные)


- CSR - три списка:
    - row (номера строк)
    - col (намера стоблцов)
    - data (данные)

Создание модели сводится к нескольким строкам.



In [132]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

model = LightFM(loss='warp')
model.fit(ratings, epochs=30, num_threads=2)
test_precision = precision_at_k(model, ratings_test, k=5).mean()

Попробуем вручную загрузить данные

В качестве основного параметра мы передали матрицу оценок. В этом случае выполняется матричная факторизация (matrix factorization).

In [131]:
test_precision

0.11049842

# <span style="color:blue">Surprise</span>

Вторая библиотека, которую мы рассмотрим, называется **Surprise**.

Варианты усатновки:
- pip install scikit-learn
- conda -c conda-forge install scikit-surprise
- через Anaconda GUI 

В бибилиотеке есть 3 тестовых датасета:
- MovieLens (100k)
- MovieLens (1m)
- Jester

Подгрузим MovieLens

In [2]:
# Load the movielens-100k dataset (download it if needed),
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')

Можно также загрузить и свой датасет. Для этого создать объект класса Reader с указанием формата файла.

In [ ]:
file_path = os.path.expanduser('ml-100k/ratings.data')
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(file_path, reader=reader)

Доступные в библиотеке Surprise <a href="https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html">алгоритмы</a>:
- Baseline, Normal, SlopeOne
- kNN
- Factorization (SVD, NMF)
- CoClustering

Опишем чуть подробнее несколько алгоритмов из этого списка: Baseline, Slope One и CoClustering

**Normal**

Прогноз оценки генерируется случайным образом из нормального распределения (чьи параметры оцениваются из данных)

**Baseline**

Каждая оценка складывается из трех коэффициентов: 

$\hat{r}_{ui} = \mu + b_u + b_i$, где

$\mu$ - глобальный средний рейтинг в системе

$b_u$ - насколько данный пользователь ставит оценки выше/ниже, чем среднестатистический юзер

$b_i$ - как высоко оценивают данный товар, точнее насколько выше/ниже своей средней оценки

Коэффициенты $b_u$ и $b_i$ получаем минимизацией функционала:

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - (\mu + b_u + b_i)\right)^2 +
\lambda \left(b_u^2 + b_i^2 \right)$

Минимизировать можно двумя методами оптимизациями: SGD и ALS.

Есть ещё приближенное решение без оптимизации

**SlopeOne**

$\hat{r}_{ui} = \mu_u + \frac{1}{
|R_i(u)|}
\sum\limits_{j \in R_i(u)} \text{dev}(i, j)$

Предположим, пользователь U оценил item1, а мы хотим получить его оценку для item2. 
Смотрим, насколько в среднем item2 оценивают выше, чем item1 и прибавляем соотвествующую дельту
Если пользователь оценил несколько товаров, то считаем такие же дельты для всех оцененных им товаров и получаем средневзвешенную оценку.

**Co-Clustering**

Ключевое предположение - оценка товара зависит во-первых от типа пользователя, во-вторых от типа продукта. 

То есть, мы выделяем скажем всего 10 типов пользователей и 10 типов продукта, и по каждому такому сочетанию ставим оценку. Процесс выделения подобных блоков в матрице оценок называется bi-clustering или co-clustering.

$\hat{r}_{ui} = \overline{C_{ui}}$, где $\overline{C_{ui}}$ - средняя оценка в ко-кластере, в который входят пользователь и товар.

Однако у нас есть еще информация по отдельным пользователям и товарам, было бы глупо её не использовать.

$\hat{r}_{ui} = \overline{C_{ui}} + (\mu_u - \overline{C_u}) + (\mu_i
- \overline{C_i})$, где C_ui - средняя оценка в ко-кластере ui, Cu -  Ci

То есть Логика точно та же, что в подходе baseline, но нормировка выполняется отдельно в рамках каждого кластера => прогноз может быть более точным

In [ ]:
from surprise import SVD
algo = SVD()

trainset = data.build_full_trainset()
algo.fit(trainset)
predictions = algo.test(testset)

Разделить на Train/Test можно методом train_test_split

In [ ]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=.25)

algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Также в Suprise есть встроенная кросс-валидация

In [ ]:
cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Также в Suprise есть встроенный GridSearchCV, что сильно упрощает оптимизацию параметров алгоритма

In [ ]:
param_grid = {'k':[5,10], 'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
for x in data.folds():
    print(x[0])

In [ ]:
# Run 5-fold cross-validation and print results
from surprise.model_selection import cross_validate
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

# <span style="color:blue">Implicit<span>

**Implicit** - библиотека, заточенная под работу с неявными рейтингами (когда оценки нету, а проявлением интереса считается просмотр, клик и прочее), отсюда название.

Варианты установки:

- pip install implicit
- conda install -c conda-forge implicit
- Anaconda GUI

In [1]:
import implicit

import os
import pandas
data_path = "/Users/Konstantin/HSE/MasterProgram/Практический Семинар/Recommender Systems/RecSys_lesson1/ml-1m/"

Загрузка тестового датасета выполняется методом **get_movielens()**. Параметр variant определяет размер загружаемой выборки. Мы будем использовать "1m" (1 млн оценок).

На выходе:
- ratings - матрица оценок (в sparse-формате)
- titles - названия товаров (фильмы)

In [37]:
from implicit.datasets.movielens import get_movielens
titles, ratings = get_movielens(variant='1m')

Приведем матрицу оценок к бинарному формату 0/1

In [39]:
import numpy
ratings.data[ratings.data < 4.0] = 0
ratings.eliminate_zeros()
ratings.data = numpy.ones(len(ratings.data))

In [18]:
ratings.shape

(3953, 6041)

В implicit есть несколько моделей:
- AlternatingLeastSquares
- BayesianPersonalizedRanking
- TFIDFRecommender
- CosineRecommender
- BM25Recommender

In [62]:
model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(ratings)

100%|██████████| 15.0/15 [00:04<00:00,  2.82it/s]


Возьмем какого-нибудь юзера и посмотрим, какие фильмы ему понравились

In [58]:
userid = 4
for y in [titles[x] for x in numpy.nonzero(ratings.T[userid,:])[1]]:
    print(y)

Star Wars: Episode IV - A New Hope (1977)
Jurassic Park (1993)
Die Hard (1988)
E.T. the Extra-Terrestrial (1982)
Raiders of the Lost Ark (1981)
Good, The Bad and The Ugly, The (1966)
Alien (1979)
Terminator, The (1984)
Jaws (1975)
Rocky (1976)
Saving Private Ryan (1998)
King Kong (1933)
Run Lola Run (Lola rennt) (1998)
Goldfinger (1964)
Fistful of Dollars, A (1964)
Thelma & Louise (1991)
Hustler, The (1961)
Mad Max (1979)


Сгенерировать реокемндации - model.recommend()

In [66]:
recommended_film_ids = model.recommend(userid=userid, user_items=ratings.T.tocsr(), N=5)
recommended_film_ids

[(589, 0.4580247),
 (2571, 0.4282091),
 (1291, 0.40621763),
 (1304, 0.38326198),
 (1196, 0.37946856)]

Переведем айдишники в названия

In [70]:
recommened_film_ids = [(titles[x[0]],x[1]) for x in recommended_film_ids]
recommened_film_ids

[('Terminator 2: Judgment Day (1991)', 0.4580247),
 ('Matrix, The (1999)', 0.4282091),
 ('Indiana Jones and the Last Crusade (1989)', 0.40621763),
 ('Butch Cassidy and the Sundance Kid (1969)', 0.38326198),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 0.37946856)]

In [75]:
numpy.ediff1d(ratings.indptr)

array([   0, 1655,  285, ...,   33,   27,  264], dtype=int32)